### Load and plot resident cell types

In [ ]:
import scanpy as sc
import numpy as np
import pandas as pd
import scvi
import seaborn as sns
import os,sys
import anndata
import scvi
import anndata as ad

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" # to show output from all the lines in a cells
pd.set_option('display.max_column',None) # display all the columns in pandas
pd.options.display.max_rows = 100

from datetime import date
today = str(date.today())
sc.logging.print_header()
## Add this line so the text on pdf is correctly recognised!!!
from matplotlib import rcParams
rcParams['pdf.fonttype'] = 42
sc.settings.set_figure_params(dpi = 150, color_map = 'RdPu', dpi_save = 300, vector_friendly = True, format = 'pdf')

In [ ]:
#model_folder = '/nfs/team205/ny1/ThymusSpatialAtlas/Figure4/scvi_models/'
figures = '/nfs/team205/ny1/ThymusSpatialAtlas/Figure_4_nat/figures/'
results = '/nfs/team205/ny1/ThymusSpatialAtlas/Figure_4_nat/'

In [ ]:
clean_folder = '/nfs/team205/ny1/ThymusSpatialAtlas/Figure_1_nat/clean'

In [ ]:
major_mrkrs = ['PTPRC', 'CD3G', 
               'HOXA9', 
               'HES1', #DN
               'RORC', #DP
               'CCR9',
              'PDCD1', #CD8aa
              'CD8A', 'CD8B', 
              'CD4', 'CD40LG', 
              'CRTAM', 'ANXA1', #memory
             'FOXP3', 'DHRS3', 
               'TRDC', #gama-delta 
              'EOMES', 'KLRD1', # NK cells
              'TNFSF11', # ILC
              'CD19', 'VPREB1', # pre-pro B cells
              'IGHD', 'IGHA1', 'IGHG1', 
              'CLEC9A', 'CLEC10A', 
              'LAMP3', 
             'LILRA4','S100A9', 'C1QA', 'TPSB2', 'ITGA2B', 'GYPA', 
              'PDGFRA', 'COLEC11', 'FBN1', 
              'RGS5', 'CDH5', 
               'LYVE1', 'EPCAM', 'PSMB11', 'DLK2', 'KRT14', 
               'CCL19', 'AIRE', 'KRT1', 'POU2F3',
               'MYOD1', 'NEUROG1']

In [ ]:
def run_scvi2(adata, batch_hv="age_group", batch_scvi="sample", \
             cat_cov_scvi=["DonorID", "10X_version", "Sex", "Age_group"], cont_cov_scvi=["percent_mito"], \
             include_genes=[], exclude_cc_genes=True, exclude_mt_genes=True, 
             exclude_vdjgenes = True, remove_cite = False, hvg = 3000, max_epochs = 350, vae_name="", **kwargs):
    adata_scvi = sc.AnnData(X=adata.layers['counts'].copy(), obs=adata.obs.copy(), var=adata.var.copy())
    #adata_scvi = adata_scvi[:,0:-150] # remove cite genes
    # adata_scvi.layers["counts"] = adata_scvi.X.copy()
    # sc.pp.normalize_total(adata_scvi, target_sum=1e4)
    # sc.pp.log1p(adata_scvi)
    # keep full dimension safe
    #    sc.pp.highly_variable_genes(adata_scvi, flavor="seurat_v3", n_top_genes=10000, layer="counts",\
    #                                batch_key="Age_group")
    if remove_cite:
        print('removing CITEseq genes pre SCVI')
        adata_scvi = adata_scvi[:,~adata_scvi.var['cite']].copy() # remove cite genes
    gene_list = adata_scvi.var_names.tolist()
    if exclude_cc_genes:
        cell_cycle_genes = [x.strip() for x in open('/nfs/team205/vk8/processed_data/regev_lab_cell_cycle_genes.txt')]
        [gene_list.remove(i) for i in cell_cycle_genes if i in gene_list]
    if exclude_mt_genes:
        mt_genes = adata.var_names[adata.var_names.str.startswith('MT-')]
        [gene_list.remove(i) for i in mt_genes if i in gene_list]
    if exclude_vdjgenes:
        import re
        [gene_list.remove(i) for i in gene_list if re.search('^TR[AB][VDJ]|^IG[HKL][VDJC]', i)]
    
    print('Removed excluded genes')
    adata_scvi = adata_scvi[:,gene_list].copy()
    sc.pp.highly_variable_genes(adata_scvi, flavor="seurat_v3", n_top_genes=hvg, batch_key=batch_hv)
    selected_genes = list(set(adata_scvi.var.loc[adata_scvi.var['highly_variable']].index.tolist() + include_genes))
    adata_scvi = adata_scvi[:, selected_genes].copy()
    print(f'Highly variable genes selected in total {adata_scvi.shape}')
    scvi.model.SCVI.setup_anndata(adata_scvi, batch_key=batch_scvi,
                                  categorical_covariate_keys=cat_cov_scvi,
                                  continuous_covariate_keys=cont_cov_scvi)
    scvi_kwargs = {k: v for k, v in kwargs.items() if k in scvi.model.SCVI.__init__.__code__.co_varnames}
    vae = scvi.model.SCVI(adata_scvi, **scvi_kwargs)
   # vae = scvi.model.SCVI(adata_scvi,n_layers=2, n_latent=30)
    train_kwargs = {k: v for k, v in kwargs.items() if k in vae.train.__code__.co_varnames}
    vae.train(**train_kwargs)
    # adata_scvi.obsm["X_scVI"] = vae.get_latent_representation()
    #   adata_raw_scvi = adata.copy()
    print('scvi model trained')
    adata_scvi.obsm["X_scVI"] = vae.get_latent_representation()
    sc.pp.neighbors(adata_scvi, use_rep="X_scVI")
    sc.tl.leiden(adata_scvi, resolution = 1, key_added = "leiden_r1.0")
    sc.tl.leiden(adata_scvi, resolution = 1.2, key_added = "leiden_r1.2")
    sc.tl.leiden(adata_scvi, resolution = 1.4, key_added = "leiden_r1.4")
    sc.tl.leiden(adata_scvi, resolution = 1.6, key_added = "leiden_r1.6")
    #   sc.tl.umap(adata_scvi)
    sc.tl.umap(adata_scvi)
    print('DR and clustering performed')
    adata_raw_scvi = adata.copy()
    adata_raw_scvi.obsm['X_scVI'] = adata_scvi.obsm['X_scVI'].copy()
    adata_raw_scvi.obsm['X_umap'] = adata_scvi.obsm['X_umap'].copy()
    adata_raw_scvi.obsp = adata_scvi.obsp.copy()
    adata_raw_scvi.uns = adata_scvi.uns.copy()
    adata_raw_scvi.obs[["leiden_r1.0", "leiden_r1.2", "leiden_r1.4", "leiden_r1.6"]] = adata_scvi.obs[["leiden_r1.0", "leiden_r1.2", "leiden_r1.4", "leiden_r1.6"]].copy()
    sc.pp.normalize_total(adata_raw_scvi, target_sum = 1e4)
    sc.pp.log1p(adata_raw_scvi)
    sc.pl.umap(
        adata_raw_scvi,
        color=['cell_type_level_4'], legend_loc = "on data", legend_fontsize = 4, frameon=False,
        ncols=2,
    )
    sc.pl.umap(
        adata_raw_scvi,
        color=['age_group', 'sex', 'donor'], frameon = False, ncols = 2)
    sc.pl.umap(
        adata_raw_scvi,
        color=['study', 'chemistry_simple', 'doublet_score'])
    #cat_cov_str = "-".join(cat_cov_scvi)
    #    vae.save(f"/nfs/team205/vk8/scripts/scvi/Results/scvi_models/{vae_name}_scvi_batch{batch_scvi}_cat{cat_cov_str}_cont{cont_cov_scvi}_10000hvgenes_{today}", save_anndata = True)
    results = {}
    results['data'] = adata_raw_scvi
    results['vae'] = vae
    return (results)

In [ ]:
markers_fb2 = ['PDGFRA','PDGFRB', "VIM",
              'COLEC11', 'ALDH1A2', 'GDF10', 'MKI67', # Perilobular fibroblasts (or Parenhcymal)
               'FBN1', 'PI16', 'MFAP5', 'SEMA3C', #'SEMA3D','DPP4',  # Capsular fibroblasts (or Interlobular, vasc associated)
              'COL9A3', 'COL13A1', 'WIF1', 'DHRS3', # Fetal-specific perilobular firboblasts
              'CCL19','IL33','TNFSF10', 'CXCL9', 'CXCL10', 'RGS5', #putative medullary fibroblasts
               'HLA-DRA', 'SBSPON', 'HEY1', 'LTBP1', # MHCIIhigh medullary firboblasts
             'CCL21', 'FDCSP', 'CXCL13', #fetal specific fibro (med)
             'TNNT1', 'NKX2-5', 'KRT8',  #fetal specific fibro (cardiac?TEC?)
              'CES1', 'RSPO2' #fetal specific fibro (mesenchyme? unclear?)
              ]

In [ ]:
markers_dict = {}

## Load Fibros

In [ ]:
adata_all = sc.read(f"{clean_folder}/adata_full_rev_4_clean.h5ad")

In [ ]:
fbs = [i for i in adata_all.obs['cell_type_level_4_explore'].cat.categories if 'FB' in i or 'Fb' in i]

In [ ]:
adata_fbs = adata_all[adata_all.obs['cell_type_level_4_explore'].isin(fbs),:].copy()

In [ ]:
adata_fbs.obs['age_group'].value_counts()

In [ ]:
adata_fbs.obs['sex'].value_counts()

In [ ]:
#adata_fbs.obs['age_group2'] = adata_fbs.obs['age_group'].apply(lambda x: 'prenatal' if x == 'fetal' else 'postnatal') 
samples = adata_fbs.obs['sample'].value_counts().index[adata_fbs.obs['sample'].value_counts()>5].tolist()

In [ ]:
adata_fbs.obs['sample'].value_counts()>5

In [ ]:
adata_fbs.layers['counts'] = adata_fbs.X.copy()

### note study is excluded from the covariates as some studies had enriched for particular fbs subtypes

In [ ]:
adata_fbs_scvi = run_scvi2(adata_fbs,  batch_hv="age_group", hvg = 5000, batch_scvi='sample', 
          cat_cov_scvi=['chemistry_simple','age_group','donor', 'sex'], cont_cov_scvi = None, include_genes=[], 
          exclude_cc_genes=True, exclude_vdjgenes=True, exclude_mt_genes = True, 
          n_layers=2, n_latent=30, max_epochs=350, batch_size=2000)

In [ ]:
adata_scvi = adata_fbs_scvi['data'].copy()

In [ ]:
sc.pl.umap(adata_scvi, color = ["leiden_r1.0", "leiden_r1.2", "leiden_r1.4", "leiden_r1.6"], 
           legend_loc = "on data", legend_fontsize = 5, frameon = False)

In [ ]:
sc.pl.umap(adata_scvi, color = major_mrkrs)

In [ ]:
sc.pl.umap(adata_scvi, color = ['PDGFRA','PDGFRB', "VIM",
              'COLEC11', 'ALDH1A2', 'GDF10', 'MKI67', # Perilobular fibroblasts (or Parenhcymal)
               'FBN1', 'PI16', 'MFAP5', 'SEMA3C', #'SEMA3D','DPP4',  # Capsular fibroblasts (or Interlobular, vasc associated)
              'COL9A3', 'COL13A1', 'WIF1', 'DHRS3', # Fetal-specific perilobular firboblasts
              'CCL19','IL33','TNFSF10', 'CXCL9', 'CXCL10', 'RGS5', #putative medullary fibroblasts
               'HLA-DRA', 'SBSPON', 'HEY1', 'LTBP1', # MHCIIhigh medullary firboblasts
             'CCL21', 'FDCSP', 'CXCL13', #fetal specific fibro (med)
             'TNNT1', 'NKX2-5', 'KRT8',  #fetal specific fibro (cardiac?TEC?)
              'CES1', 'RSPO2' #fetal specific fibro (mesenchyme? unclear?)
              ])

In [ ]:
fbs_mrkrs = {'General': ['PDGFRA','PDGFRB', "VIM"],
             'PeriLobular': ['COLEC11', 'ALDH1A2', 'GDF10', 'MKI67'],
             'InterLobular': ['FBN1', 'PI16', 'MFAP5', 'SEMA3C'],  #'SEMA3D','DPP4',
             'Fetal(subcapsular)': ['COL9A3', 'COL13A1', 'WIF1', 'DHRS3'], 
             'Medullary': ['CCL19','IL33','TNFSF10', 'CXCL9', 'CXCL10', 'RGS5'],
             'Medullary APC': ['HLA-DRA', 'SBSPON', 'HEY1', 'LTBP1'],
             'Fetal(lymphoid org)': ['CCL21', 'FDCSP', 'CXCL13'], 
             'Fetal(muscle)': ['TNNT1', 'PAX7', 'MYF5'],
             'Fetal(RSPO2)': ['CES1', 'RSPO2']
             }

In [ ]:
mhc1 = ['HLA-A', 'HLA-B', 'HLA-C', 'HLA-E', 'HLA-F', 'HLA-G']
mhc2 = ['HLA-DMA', 'HLA-DMB', 'HLA-DOA', 'HLA-DOB', 'HLA-DPA1','HLA-DPB1',  'HLA-DQA1', 'HLA-DQA2',
 'HLA-DQB1', 'HLA-DQB1-AS1','HLA-DQB2', 'HLA-DRA', 'HLA-DRB1', 'HLA-DRB5', ]

In [ ]:
sc.pl.umap(adata_scvi, color = mhc1)
sc.pl.umap(adata_scvi, color = mhc2)

In [ ]:
fbs_subtypes = ['PeriloFb', 'PeriloFb-Prolif', 'InterloFb', 'InterloFb-COL9A3', 'medFB-MHCIIh', 'medFb',
       'medFb-RGS5', 'fetFB-CCL21', 'fetFB-NKX2-5', 'fetFB-RSPO2']

In [ ]:
def population_plots(adata_scvi, mrkrs_dict, save_name, figures, cat_order, anno = 'cell_type_level_4_upd', 
                     anno_cov = ['study', 'sex', 'age_group', 'chemistry_simple']):
    adata_scvi.obs[anno] = adata_scvi.obs[anno].cat.reorder_categories(cat_order).copy()
    sc.pl.umap(adata_scvi, color = anno, legend_loc = "on data", legend_fontsize = 5, 
               frameon = False, save = f"{save_name}_scvi_celltype.pdf")
    sc.pl.umap(adata_scvi, color = anno_cov, wspace = 0.3, frameon = False, 
               save = f"{save_name}_scvi_covariates.pdf")
    dp = sc.pl.dotplot(adata_scvi, groupby = anno, var_names = mrkrs_dict, 
                      standard_scale = "var", return_fig = True)
    dp = dp.add_totals().style(dot_edge_color='black', dot_edge_lw=0.5, cmap = "Reds")
    # Save the figure
    dp.savefig(f"{figures}{save_name}_HTSA_scvi_mrkrs.pdf")

In [ ]:
population_plots(adata_scvi, mrkrs_dict = fbs_mrkrs, save_name = "FB", 
                 figures = '/nfs/team205/ny1/ThymusSpatialAtlas/Figure_4_nat/figures/', 
                 cat_order = ['PeriloFb', 'PeriloFb-Prolif', 'InterloFb', 'InterloFb-COL9A3','medFb',
                              'medFb-RGS5', 'medFB-MHCIIh', 'fetFB-CCL21', 'fetFB-NKX2-5', 'fetFB-RSPO2'],
                 anno = 'cell_type_level_4_explore', anno_cov = ['study', 'sex', 'age_group', 'chemistry_simple'])

In [ ]:
adata_scvi.write(f'{clean_folder}/fbs_scvi_updated_{today}.h5ad')

adata_fbs_scvi['vae'].save(f'{clean_folder}/models/fbs_scvi_updated_{today}.h5ad')